# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

##### Setting Pandas float format to 2 decimal places so that all scores and percentage are automatically displayed with 2 decimal places. No need to format in further steps - `` pd.set_option('display.float_format','{:.2f}'.format) ``

In [1]:
# Dependencies and Setup
import pandas as pd
pd.set_option('display.float_format','{:.2f}'.format)

# File to Load 
school_data_file = "Resources/schools_complete.csv"
student_data_file = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_file)
student_data_df = pd.read_csv(student_data_file)

# Combine the data into a single dataset.  
school_data_merged_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

#### Total number of unique schools

In [2]:
# Calculate the total number of unique schools
school_count = school_data_merged_df['school_name'].nunique()

school_count

15

#### Total number of students

In [3]:
# Calculate the total number of students
student_count = len(student_data_df[['Student ID','student_name','school_name']].drop_duplicates())
student_count

39170

#### Total budget

In [4]:
# Calculate the total budget
total_budget = school_data_merged_df[['school_name','budget']].drop_duplicates()
total_budget = total_budget['budget'].sum()
total_budget

24649428

#### Average math and reading scores

In [5]:
# Calculate the average (mean) reading and math scores
average_scores = school_data_merged_df[['math_score','reading_score']].mean()
average_scores


math_score      78.99
reading_score   81.88
dtype: float64

#### Convert the average score series to dataframe and rename index to score_type

In [6]:
df_scores = pd.DataFrame(average_scores).reset_index()
df_scores = df_scores.rename(columns={'index':'score_type',0 :'average_score'})
df_scores


,score_type,average_score
0,math_score,78.99
1,reading_score,81.88


#### Average Math score

In [7]:
# Calculates average scores by type: math score
input_score_type = 'math_score'
average_math_score = df_scores[df_scores['score_type']==input_score_type]
#average_math_score = round(average_math_score,2)
average_math_score = list(average_math_score['average_score'])[0]
average_math_score

78.98537145774827

#### Average Reading score

In [8]:
# Calculate the average scores by type: reading score
input_score_type = 'reading_score'
average_reading_score = df_scores[df_scores['score_type']==input_score_type]
#average_reading_score = round(average_reading_score,2)
average_reading_score = list(average_reading_score['average_score'])[0]
average_reading_score

81.87784018381414

#### Percentage of students passing in math(math score greater than equal to 70)

In [9]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
pass_condition_df = student_data_df[student_data_df['math_score'] >= 70]
math_passing_count = len(pass_condition_df)
#pass_condition_df.count()['student_name'] #alternate in pandas
total_student_count = len(student_data_df)
passing_math_percentage = math_passing_count / float(total_student_count) * 100
#passing_math_percentage = round(passing_math_percentage,2)
passing_math_percentage

74.9808526933878

#### Percentage of students passing in reading(reading score greater than equal to 70)

In [10]:
# Calculate the percentage of students who passeed reading 
passing_reading_count = len(student_data_df[student_data_df['reading_score'] >= 70])
passing_reading_percentage = passing_reading_count / float(total_student_count) * 100
#passing_reading_percentage = round(passing_reading_percentage,2)
passing_reading_percentage

85.80546336482001

#### Percentage of students passing in math AND reading

In [11]:
passing_math_reading_count = len(student_data_df[(student_data_df['math_score'] >= 70) & (student_data_df['reading_score'] >= 70)])
overall_passing_rate = passing_math_reading_count / float(total_student_count) * 100
#overall_passing_rate = round(overall_passing_rate,2)
overall_passing_rate

65.17232575950983

## District Summary Output

In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
#Creating a dictionary with the metric values and formatting
district_summary = {'Total Schools': school_count, 'Total Students':student_count,'Total Budget':total_budget,
                   'Average Math Score':average_math_score,'Average Reading Score': average_reading_score,
                   'Math Passing Percent': '{0} %'.format(str(passing_math_percentage)),
                    'Reading Passing Percent': '{0} %'.format(str(passing_reading_percentage)),
                   'Overall Passing Rate':'{0} %'.format(str(overall_passing_rate))}

#Convert dictionary into dataframe
district_summary = pd.DataFrame([district_summary])
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Passing Percent,Reading Passing Percent,Overall Passing Rate
0,15,39170,24649428,78.99,81.88,74.9808526933878 %,85.80546336482001 %,65.17232575950983 %


# School Summary

In [13]:
#passing required fields
fieldnames = ['school_name','type','Student ID','budget','math_score','reading_score']

#Student counts by school and type
student_counts_df = school_data_merged_df[fieldnames].groupby(['school_name','type'])['Student ID'].count()

#Converting to dataframes
student_counts_df = pd.DataFrame(student_counts_df)

student_counts_df.head()

,,Student ID
school_name,type,
Bailey High School,District,4976
Cabrera High School,Charter,1858
Figueroa High School,District,2949
Ford High School,District,2739
Griffin High School,Charter,1468


In [14]:
# Calculating average math and reading scores by school and type
average_scores_df = school_data_merged_df[fieldnames].groupby(['school_name','type'])[['math_score','reading_score']].mean()
average_scores_df= pd.DataFrame(average_scores_df)

# Calculating Budget per school
total_school_budget_df = school_data_merged_df[fieldnames].groupby(['school_name','type'])['budget'].mean()
total_school_budget_df = pd.DataFrame(total_school_budget_df)

#Combining school budget with student counts dataframe
#joining on indexes
school_budget_students_df = pd.merge(student_counts_df,total_school_budget_df, how='inner',left_index=True,right_index=True)

#Per student budget(performing calculations within a dataframe)
school_budget_students_df['Per Student Budget'] = school_budget_students_df['budget'] / school_budget_students_df['Student ID']
school_budget_students_df

,,Student ID,budget,Per Student Budget
school_name,type,,,
Bailey High School,District,4976,3124928,628.00
Cabrera High School,Charter,1858,1081356,582.00
Figueroa High School,District,2949,1884411,639.00
Ford High School,District,2739,1763916,644.00
Griffin High School,Charter,1468,917500,625.00
Hernandez High School,District,4635,3022020,652.00
Holden High School,Charter,427,248087,581.00
Huang High School,District,2917,1910635,655.00
Johnson High School,District,4761,3094650,650.00


In [15]:
#Calculating the number of students passing in math
passing_math_score_df = school_data_merged_df[school_data_merged_df['math_score']>=70][['Student ID','school_name','type']]

#Number of students passing in math per school and type
passing_math_score_df = passing_math_score_df.groupby(['school_name','type']).count()


#renaming the Student ID column to Passing Student Count in math
passing_math_score_df = passing_math_score_df.rename(columns = {'Student ID': 'Passing_Student_Math' })

#Calculating the number of students passing in reading
passing_reading_score_df = school_data_merged_df[school_data_merged_df['reading_score']>=70][['Student ID','school_name','type']]

#Number of students passing in reading per school and type
passing_reading_score_df = passing_reading_score_df.groupby(['school_name','type']).count()


#renaming the Student ID column to represent Passing Student Count in reading
passing_reading_score_df = passing_reading_score_df.rename(columns = {'Student ID': 'Passing_Student_Reading' })


#overall
overall_passing_score_df = school_data_merged_df[(school_data_merged_df['math_score']>=70) & (school_data_merged_df['reading_score']>=70)][['Student ID','school_name','type']]

#Number of students passing in reading and math per school and type
overall_passing_score_df = overall_passing_score_df.groupby(['school_name','type']).count()


#renaming the Student ID column to represent Overall count for students who passed in both math and reading
overall_passing_score_df = overall_passing_score_df.rename(columns = {'Student ID': 'Overall_Passing_Student' })

overall_passing_score_df

,,Overall_Passing_Student
school_name,type,
Bailey High School,District,2719
Cabrera High School,Charter,1697
Figueroa High School,District,1569
Ford High School,District,1487
Griffin High School,Charter,1330
Hernandez High School,District,2481
Holden High School,Charter,381
Huang High School,District,1561
Johnson High School,District,2549


In [16]:
#Combining school budget, student counts, average math and reading scores
#joining on indexes
school_budget_students_df = pd.merge(school_budget_students_df,average_scores_df,
                                    how='inner',left_index=True,right_index=True)
school_budget_students_df

,,Student ID,budget,Per Student Budget,math_score,reading_score
school_name,type,,,,,
Bailey High School,District,4976,3124928,628.00,77.05,81.03
Cabrera High School,Charter,1858,1081356,582.00,83.06,83.98
Figueroa High School,District,2949,1884411,639.00,76.71,81.16
Ford High School,District,2739,1763916,644.00,77.10,80.75
Griffin High School,Charter,1468,917500,625.00,83.35,83.82
Hernandez High School,District,4635,3022020,652.00,77.29,80.93
Holden High School,Charter,427,248087,581.00,83.80,83.81
Huang High School,District,2917,1910635,655.00,76.63,81.18
Johnson High School,District,4761,3094650,650.00,77.07,80.97


In [17]:
#Combining school budget, student counts, average math, average reading scores,students passing in math

school_budget_students_df_out = pd.merge(school_budget_students_df,passing_math_score_df,how = 'left',left_index=True,right_index=True)


#Combining school budget, student counts, average math, average reading scores,students passing in reading

school_budget_students_df_out = pd.merge(school_budget_students_df_out,passing_reading_score_df,how = 'left',
                                     left_index=True,right_index=True)



In [18]:
#  Combining the overall passing count with the school_budget_students_df_out

school_budget_students_df_out = pd.merge(school_budget_students_df_out,overall_passing_score_df,how = 'left',left_index=True,right_index=True)
school_budget_students_df_out

,,Student ID,budget,Per Student Budget,math_score,reading_score,Passing_Student_Math,Passing_Student_Reading,Overall_Passing_Student
school_name,type,,,,,,,,
Bailey High School,District,4976,3124928,628.00,77.05,81.03,3318,4077,2719
Cabrera High School,Charter,1858,1081356,582.00,83.06,83.98,1749,1803,1697
Figueroa High School,District,2949,1884411,639.00,76.71,81.16,1946,2381,1569
Ford High School,District,2739,1763916,644.00,77.10,80.75,1871,2172,1487
Griffin High School,Charter,1468,917500,625.00,83.35,83.82,1371,1426,1330
Hernandez High School,District,4635,3022020,652.00,77.29,80.93,3094,3748,2481
Holden High School,Charter,427,248087,581.00,83.80,83.81,395,411,381
Huang High School,District,2917,1910635,655.00,76.63,81.18,1916,2372,1561
Johnson High School,District,4761,3094650,650.00,77.07,80.97,3145,3867,2549


In [19]:
# Calculating % passing in math and reading

school_budget_students_df_out['%passing math'] = school_budget_students_df_out['Passing_Student_Math'] / school_budget_students_df_out['Student ID'] * 100
school_budget_students_df_out['%passing reading'] = school_budget_students_df_out['Passing_Student_Reading'] / school_budget_students_df_out['Student ID'] * 100

#Calculating % overall passing in both math and reading
school_budget_students_df_out['%overall passing'] = school_budget_students_df_out['Overall_Passing_Student'] / school_budget_students_df_out['Student ID'] * 100

school_budget_students_df_out

,,Student ID,budget,Per Student Budget,math_score,reading_score,Passing_Student_Math,Passing_Student_Reading,Overall_Passing_Student,%passing math,%passing reading,%overall passing
school_name,type,,,,,,,,,,,
Bailey High School,District,4976,3124928,628.00,77.05,81.03,3318,4077,2719,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.00,83.06,83.98,1749,1803,1697,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.00,76.71,81.16,1946,2381,1569,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.00,77.10,80.75,1871,2172,1487,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.00,83.35,83.82,1371,1426,1330,93.39,97.14,90.60
Hernandez High School,District,4635,3022020,652.00,77.29,80.93,3094,3748,2481,66.75,80.86,53.53
Holden High School,Charter,427,248087,581.00,83.80,83.81,395,411,381,92.51,96.25,89.23
Huang High School,District,2917,1910635,655.00,76.63,81.18,1916,2372,1561,65.68,81.32,53.51
Johnson High School,District,4761,3094650,650.00,77.07,80.97,3145,3867,2549,66.06,81.22,53.54


## School Summary Output

In [20]:
#renaming columns using column mapping dictionary

column_map ={'Student ID': 'Total students',
            'budget': 'Total school budget',
            'math_score': 'Average math score',
            'reading_score':'Average reading score'}

per_school_summary_df = school_budget_students_df_out.rename(columns=column_map)

#dropping unwanted columns
per_school_summary_df = per_school_summary_df.drop(columns=['Passing_Student_Math','Passing_Student_Reading','Overall_Passing_Student'])

per_school_summary_df

,,Total students,Total school budget,Per Student Budget,Average math score,Average reading score,%passing math,%passing reading,%overall passing
school_name,type,,,,,,,,
Bailey High School,District,4976,3124928,628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,1081356,582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,1884411,639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,1763916,644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,917500,625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,3022020,652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,248087,581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,1910635,655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,3094650,650.00,77.07,80.97,66.06,81.22,53.54


## Highest-Performing Schools (by % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary_df.sort_values(['%overall passing'],ascending=False)
top_schools.head(5)

,,Total students,Total school budget,Per Student Budget,Average math score,Average reading score,%passing math,%passing reading,%overall passing
school_name,type,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,1043130,638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,917500,625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,1319574,578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,585858,609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary_df.sort_values(['%overall passing'],ascending=True)
bottom_schools.head(5)

,,Total students,Total school budget,Per Student Budget,Average math score,Average reading score,%passing math,%passing reading,%overall passing
school_name,type,,,,,,,,
Rodriguez High School,District,3999,2547363,637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,1884411,639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,1910635,655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,3022020,652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,3094650,650.00,77.07,80.97,66.06,81.22,53.54


## Math and Reading Score by Grade (Top 30)

In [23]:
#using the student_data_df(student data) for this calculation
student_data_df.head()

#listing the grades
grades =['9th','10th','11th','12th']

#filtering based on grades and sorting based on school name
student_data_df_filtered = student_data_df[student_data_df['grade'].isin(grades)].sort_values(['school_name'])


#Calculating average reading and math score for 9th,10th,11th and 12th grade students in each school.
scores_by_grade_df = student_data_df_filtered.groupby(['school_name','grade'])[['reading_score','math_score']].mean()

#renaming the score columns
scores_by_grade_df = scores_by_grade_df.rename(columns={'reading_score':'Average Reading Score',
                                  'math_score':'Average Math Score'})

#display the average scores(reading and math) for 9th,10th,11th and 12th grade students in each school.
scores_by_grade_df.head(30)


Average Reading Score  Average Math Score
school_name           grade                                           
Bailey High School    10th                   80.91               77.00
                      11th                   80.95               77.52
                      12th                   80.91               76.49
                      9th                    81.30               77.08
Cabrera High School   10th                   84.25               83.15
                      11th                   83.79               82.77
                      12th                   84.29               83.28
                      9th                    83.68               83.09
Figueroa High School  10th                   81.41               76.54
                      11th                   80.64               76.88
                      12th                   81.38               77.15
                      9th                    81.20               76.40
Ford High School      10th                   81.26               77.67
                      11th                   80.40               76.92
                      12th                   80.66               76.18
                      9th                    80.63               77.36
Griffin High School   10th                   83.71               84.23
                      11th                   84.29               83.84
                      12th                   84.01               83.36
                      9th                    83.37               82.04
Hernandez High School 10th                   80.66               77.34
                      11th                   81.40               77.14
                      12th                   80.86               77.19
                      9th                    80.87               77.44
Holden High School    10th                   83.32               83.43
                      11th                   83.82               85.00
                      12th                   84.70               82.86
                      9th                    83.68               83.79
Huang High School     10th                   81.51               75.91
                      11th                   81.42               76.45

## Scores by School Spending

#### Calculating Spending range using Bins

In [24]:
# Establish the bins 
spending_bins = [0, 585.0, 630.0, 645.0, 680.0]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

#Using pd.cut to categorize spending based on the bins.
per_school_summary_df['Spending Ranges(per student)'] = pd.cut(x = per_school_summary_df['Per Student Budget'], bins = spending_bins, labels = labels, include_lowest = True)

#Calculating mean scores per spending range.
school_spending_df = per_school_summary_df.groupby(['Spending Ranges(per student)']).mean()

#dropping unwanted columns
spending_summary_df = school_spending_df.drop(columns=['Total school budget','Per Student Budget'])

#converting the total studenst to integer value
spending_summary_df['Total students']= spending_summary_df['Total students'].astype(int)

#display spending summary dataframe
spending_summary_df

,Total students,Average math score,Average reading score,%passing math,%passing reading,%overall passing
Spending Ranges(per student),,,,,,
<$585,1592,83.46,83.93,93.46,96.61,90.37
$585-630,2291,81.90,83.16,87.13,92.72,81.42
$630-645,2830,78.52,81.62,73.48,84.39,62.86
$645-680,4104,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

#### Calculating School Size using Bins

In [25]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Using pd.cut to categorize Scores by school size.
per_school_summary_df['School size'] = pd.cut(x = per_school_summary_df['Total students'], bins = size_bins, labels = labels, include_lowest = True)

size_summary_df = per_school_summary_df[['School size','Average math score','Average reading score','%passing math','%passing reading','%overall passing']].groupby(['School size']).mean()

#display all the scores by school size
size_summary_df

,Average math score,Average reading score,%passing math,%passing reading,%overall passing
School size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


# Scores by School Type

In [26]:
#reseting index to re-index upon school type
type_summary_df = per_school_summary_df.reset_index()

#renaming column 'type' to School Type
type_summary_df = type_summary_df.rename(columns={'type':'School Type'})

#Calculating performance based on School Type
type_summary_df = type_summary_df[['School Type','Average math score','Average reading score','%passing math','%passing reading','%overall passing']].groupby(['School Type']).mean()

type_summary_df

,Average math score,Average reading score,%passing math,%passing reading,%overall passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
